In [1]:
import os
import json
import base64
import sys
from requests import get, post
import pandas as pd
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [3]:
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes),'utf-8')

    url = 'https://accounts.spotify.com/api/token' #post request url

    headers = { # post request headers
        "Authorization": "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded" 
    } 

    data = {'grant_type': 'client_credentials'} #post request body

    result = post(url, headers=headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {'Authorization': 'Bearer ' + token}

def get_artist_pop(token, query):
    url = f'https://api.spotify.com/v1/search?query={query}&type=artist&limit=1'
    header = get_auth_header(token)
    result = get(url, headers = header)
    print(result)
    if result.status_code == 200:
        json_result = json.loads(result.content)
        popularity = json_result['artists']['items'][0]['popularity']
        return(popularity)
    else:
        return None

In [5]:
df = pd.read_csv('/Users/colemak/Documents/spotifymodel/notebook/spotify.csv')
df = df.dropna()
df['artist'] = df['artists'].apply(lambda x: x.split(';')[0])
artist_columns = df['artist'].unique()
length = len(artist_columns)
artist_pop_dict = {}
token = None
for i, artist in enumerate(artist_columns):
    if i % 5000 == 0:
        token = get_token()
    if i % 350 == 0:
        time.sleep(30)
    artist_1 = artist.split(';')[0]
    popularity = get_artist_pop(token, artist_1)
    artist_pop_dict[artist] = popularity
    print(f'{i}/{length} | {artist}:{popularity} length of dict:{len(artist_pop_dict)}')

<Response [200]>
0/17648 | Gen Hoshino:62 length of dict:1
<Response [200]>
1/17648 | Ben Woodward:45 length of dict:2
<Response [200]>
2/17648 | Ingrid Michaelson:58 length of dict:3
<Response [200]>
3/17648 | Kina Grannis:60 length of dict:4
<Response [200]>
4/17648 | Chord Overstreet:62 length of dict:5
<Response [200]>
5/17648 | Tyrone Wells:47 length of dict:6
<Response [200]>
6/17648 | A Great Big World:59 length of dict:7
<Response [200]>
7/17648 | Jason Mraz:72 length of dict:8
<Response [200]>
8/17648 | Ross Copperman:43 length of dict:9
<Response [200]>
9/17648 | Zack Tabudlo:68 length of dict:10
<Response [200]>
10/17648 | Dan Berk:44 length of dict:11
<Response [200]>
11/17648 | Anna Hamilton:40 length of dict:12
<Response [200]>
12/17648 | Landon Pigg:39 length of dict:13
<Response [200]>
13/17648 | Andrew Foy:44 length of dict:14
<Response [200]>
14/17648 | Boyce Avenue:69 length of dict:15
<Response [200]>
15/17648 | Brandi Carlile:64 length of dict:16
<Response [200]>
1

KeyboardInterrupt: 